# 🎵 Energy Prediction Model – Usage Notebook

This notebook demonstrates how to:
- Load the trained model
- Inspect key model information
- Run predictions using manual inputs
- Interactively test the model via a guided input form

No training required – inference only.


In [1]:
import pickle
import pandas as pd
import numpy as np

# Load trained model package
with open('models/model.pkl', 'rb') as f:
    model_package = pickle.load(f)

model = model_package['model']
preprocessor = model_package['preprocessor']
top_6_indices = model_package['top_6_indices']
feature_names = model_package['feature_names']

print("✅ Model loaded successfully")


✅ Model loaded successfully


In [2]:
print("MODEL INFORMATION")
print("-" * 40)
print(f"Model type     : {model_package['model_type']}")

# Get R² from grid search results (Random Forest test R²)
if 'grid_search_info' in model_package:
    grid_info = model_package['grid_search_info']
    test_r2 = grid_info.get('test_r2', 'N/A')
    cv_r2 = grid_info.get('best_cv_r2', 'N/A')
    print(f"R² Score: {test_r2:.4f}")
else:
    # Fallback to overall R² if grid search info not available
    print(f"Overall R²    : {model_package.get('r2_score', 'N/A'):.4f}")

print(f"Features used : {len(top_6_indices)}")


MODEL INFORMATION
----------------------------------------
Model type     : RandomForestRegressor
R² Score: 0.7131
Features used : 6


In [3]:
print("""
MANUAL INPUT INSTRUCTIONS (CODE-BASED)
--------------------------------------------------
Please provide ONE song with the following features.
All values must be numeric and unscaled.

Expected feature meanings and ranges (approx.):

danceability      : 0.0 – 1.0
valence           : 0.0 – 1.0
loudness          : -60 – 0        (in dB)
tempo             : 50 – 200       (BPM)
speechiness       : 0.0 – 1.0
acousticness      : 0.0 – 1.0
instrumentalness  : 0.0 – 1.0
liveness          : 0.0 – 1.0
dynamic_range     : 0.0 – 20.0     (typically 5-15)
rhythmic_complexity: 0.0 – 100.0   (typically 20-80)

You can think of this as describing a song manually.
--------------------------------------------------
""")



MANUAL INPUT INSTRUCTIONS (CODE-BASED)
--------------------------------------------------
Please provide ONE song with the following features.
All values must be numeric and unscaled.

Expected feature meanings and ranges (approx.):

danceability      : 0.0 – 1.0
valence           : 0.0 – 1.0
loudness          : -60 – 0        (in dB)
tempo             : 50 – 200       (BPM)
speechiness       : 0.0 – 1.0
acousticness      : 0.0 – 1.0
instrumentalness  : 0.0 – 1.0
liveness          : 0.0 – 1.0
dynamic_range     : 0.0 – 20.0     (typically 5-15)
rhythmic_complexity: 0.0 – 100.0   (typically 20-80)

You can think of this as describing a song manually.
--------------------------------------------------



In [4]:
# Manually define a song (example: energetic electronic track)
manual_input = {
    'danceability': 0.75,
    'valence': 0.65,
    'loudness': -6.0,
    'tempo': 128,
    'speechiness': 0.05,
    'acousticness': 0.02,
    'instrumentalness': 0.4,
    'liveness': 0.1,
    'dynamic_range': 10.0,          # Added: typical range 5-15
    'rhythmic_complexity': 50.0      # Added: typical range 20-80
}

manual_df = pd.DataFrame([manual_input])
manual_df


,danceability,valence,loudness,tempo,speechiness,acousticness,instrumentalness,liveness,dynamic_range,rhythmic_complexity
0,0.75,0.65,-6.0,128,0.05,0.02,0.4,0.1,10.0,50.0


In [5]:
manual_df['loudness_tempo'] = manual_df['loudness'] * manual_df['tempo']
manual_df['danceability_valence'] = manual_df['danceability'] * manual_df['valence']
manual_df['loudness_danceability'] = manual_df['loudness'] * manual_df['danceability']
manual_df['tempo_valence'] = manual_df['tempo'] * manual_df['valence']


In [6]:
# Preprocess the input (applies imputation and scaling)
processed = preprocessor.transform(manual_df)

# Select top 6 features (as used in training)
processed_top6 = processed[:, top_6_indices]

# Predict
prediction = model.predict(processed_top6)[0]

print(f"🎯 Predicted energy value: {prediction:.3f}")

if 0 <= prediction <= 1:
    print("✅ Prediction within valid range [0, 1]")
else:
    print("⚠️ Prediction outside expected range")
    print("   Consider clipping to [0, 1] if needed")


🎯 Predicted energy value: 0.774
✅ Prediction within valid range [0, 1]


In [7]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Create a nice header
header = HTML("""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
            padding: 20px; border-radius: 10px; margin-bottom: 20px; color: white;">
    <h2 style="margin: 0; color: white;">🎵 Energy Prediction Model - Interactive Interface</h2>
    <p style="margin: 10px 0 0 0; opacity: 0.9;">
        Adjust the sliders below to describe a song, then click "Predict Energy" to get the predicted energy value (0-1).
    </p>
</div>
""")
display(header)


In [9]:
# Create interactive widgets with proper syntax and helpful descriptions
danceability = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    description='💃 Danceability:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
danceability_help = widgets.HTML("<small><em>How suitable a track is for dancing (0=not danceable, 1=very danceable)</em></small>")

valence = widgets.FloatSlider(
    value=0.5, min=0.0, max=1.0, step=0.01,
    description='😊 Valence:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
valence_help = widgets.HTML("<small><em>Musical positiveness (0=sad/negative, 1=happy/positive)</em></small>")

loudness = widgets.FloatSlider(
    value=-10.0, min=-60.0, max=0.0, step=1.0,
    description='🔊 Loudness:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
loudness_help = widgets.HTML("<small><em>Overall loudness in decibels (dB). Typically ranges from -60 to 0</em></small>")

tempo = widgets.IntSlider(
    value=120, min=50, max=200, step=1,
    description='🎵 Tempo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
tempo_help = widgets.HTML("<small><em>Overall estimated tempo in beats per minute (BPM). Typical range: 50-200</em></small>")

speechiness = widgets.FloatSlider(
    value=0.05, min=0.0, max=1.0, step=0.01,
    description='🗣️ Speechiness:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
speechiness_help = widgets.HTML("<small><em>Presence of spoken words (0=music, 1=speech/rap)</em></small>")

acousticness = widgets.FloatSlider(
    value=0.1, min=0.0, max=1.0, step=0.01,
    description='🎸 Acousticness:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
acousticness_help = widgets.HTML("<small><em>Confidence measure of whether track is acoustic (0=electronic, 1=acoustic)</em></small>")

instrumentalness = widgets.FloatSlider(
    value=0.0, min=0.0, max=1.0, step=0.01,
    description='🎹 Instrumentalness:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
instrumentalness_help = widgets.HTML("<small><em>Predicts whether track contains vocals (0=vocals, 1=instrumental)</em></small>")

liveness = widgets.FloatSlider(
    value=0.1, min=0.0, max=1.0, step=0.01,
    description='🎤 Liveness:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
liveness_help = widgets.HTML("<small><em>Detects presence of audience in recording (0=studio, 1=live)</em></small>")

dynamic_range = widgets.FloatSlider(
    value=10.0, min=0.0, max=20.0, step=0.5,
    description='📊 Dynamic Range:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
dynamic_range_help = widgets.HTML("<small><em>Difference between loudest and quietest parts (typical: 5-15)</em></small>")

rhythmic_complexity = widgets.FloatSlider(
    value=50.0, min=0.0, max=100.0, step=1.0,
    description='🥁 Rhythmic Complexity:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='100%')
)
rhythmic_complexity_help = widgets.HTML("<small><em>Complexity of the rhythm pattern (typical: 20-80)</em></small>")


In [10]:
# Create output widget for displaying results
output = widgets.Output()

def predict_energy(_):
    """Predict energy based on user input from sliders"""
    with output:
        output.clear_output(wait=True)
        
        # Create input DataFrame
        user_input = pd.DataFrame([{
            'danceability': danceability.value,
            'valence': valence.value,
            'loudness': loudness.value,
            'tempo': float(tempo.value),
            'speechiness': speechiness.value,
            'acousticness': acousticness.value,
            'instrumentalness': instrumentalness.value,
            'liveness': liveness.value,
            'dynamic_range': dynamic_range.value,
            'rhythmic_complexity': rhythmic_complexity.value
        }])

        # Feature engineering (same as in training)
        user_input['loudness_tempo'] = user_input['loudness'] * user_input['tempo']
        user_input['danceability_valence'] = user_input['danceability'] * user_input['valence']
        user_input['loudness_danceability'] = user_input['loudness'] * user_input['danceability']
        user_input['tempo_valence'] = user_input['tempo'] * user_input['valence']

        try:
            # Preprocess
            processed = preprocessor.transform(user_input)
            
            # Select top 6 features
            processed_top6 = processed[:, top_6_indices]

            # Predict
            prediction = model.predict(processed_top6)[0]
            
            # Clamp prediction to valid range
            prediction = max(0.0, min(1.0, prediction))
            
            # Display result with nice formatting
            print("=" * 60)
            print("🎵 PREDICTION RESULT")
            print("=" * 60)
            print(f"\n✨ Predicted Energy: {prediction:.4f}")
            print(f"   (Range: 0.0 = low energy, 1.0 = high energy)")
            
            # Add interpretation
            if prediction < 0.3:
                energy_level = "🔋 Low Energy"
            elif prediction < 0.6:
                energy_level = "⚡ Medium Energy"
            else:
                energy_level = "🔥 High Energy"
            
            print(f"\n📊 Interpretation: {energy_level}")
            print("=" * 60)
            
        except Exception as e:
            print(f"❌ Error during prediction: {str(e)}")
            import traceback
            traceback.print_exc()


In [11]:
# Create predict button with nice styling
predict_button = widgets.Button(
    description="🚀 Predict Energy",
    button_style='success',
    layout=widgets.Layout(width='200px', height='50px', margin='20px auto'),
    style={'font_weight': 'bold', 'font_size': '16px'}
)
predict_button.on_click(predict_energy)

# Create sections for better organization
section1 = widgets.VBox([
    widgets.HTML("<h3 style='color: #667eea; margin-top: 20px;'>Core Musical Features</h3>"),
    danceability, danceability_help,
    widgets.HTML("<br>"),
    valence, valence_help,
    widgets.HTML("<br>"),
    loudness, loudness_help,
    widgets.HTML("<br>"),
    tempo, tempo_help,
])

section2 = widgets.VBox([
    widgets.HTML("<h3 style='color: #667eea; margin-top: 20px;'>Audio Characteristics</h3>"),
    speechiness, speechiness_help,
    widgets.HTML("<br>"),
    acousticness, acousticness_help,
    widgets.HTML("<br>"),
    instrumentalness, instrumentalness_help,
    widgets.HTML("<br>"),
    liveness, liveness_help,
])

section3 = widgets.VBox([
    widgets.HTML("<h3 style='color: #667eea; margin-top: 20px;'>Advanced Features</h3>"),
    dynamic_range, dynamic_range_help,
    widgets.HTML("<br>"),
    rhythmic_complexity, rhythmic_complexity_help,
])

# Create main interface
main_interface = widgets.VBox([
    section1,
    widgets.HTML("<hr style='margin: 20px 0;'>"),
    section2,
    widgets.HTML("<hr style='margin: 20px 0;'>"),
    section3,
    widgets.HTML("<hr style='margin: 20px 0;'>"),
    widgets.HBox([predict_button], layout=widgets.Layout(justify_content='center')),
    output
], layout=widgets.Layout(width='100%', padding='20px'))

display(main_interface)
